# International Export Shipment Data Analysis

## Introduction

This project explores an International Export Shipment dataset to understand <mark>export patterns, shipping routes, product categories, costs, and delivery performance</mark>. Through exploratory data analysis (EDA), we identify key trends, detect outliers, analyze relationships between shipment features, and gain insights that can help improve export planning and logistics decision-making.

### Why did we choose this dataset?

We selected this export-shipment dataset because it reflects a <mark>real-world business problem</mark> that companies face every day: managing international shipments efficiently.

## Objectives

In this project, we aim to explore:

### Shipment Trends

* Which products are exported the most

* Which countries are the top export destinations

### Cost & Weight Patterns

* How shipment weight affects shipping cost

* Which shipments are unusually expensive or heavy (outliers)

### Delivery Performance

* Average delivery time

* What factors contribute to delays

### Logistics Insights

* Most common transport modes

* Route efficiency and cost differences

### Importing the libraries 

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Loading the datset

df=pd.read_csv(r"C:\Users\Lenovo\Data Analytics\Dataset\export_shipment_dataset.csv")
df

### Understanding the dataset

df.columns

### Understanding the columns

* Shipment_ID
* Date
* Country
* Port_of_Loading
* Commodity
* Quantity_Tons
* Export_Value_USD
* Shipping_Cost_USD
* Transport_Mode
* Delivery_Days
* **Trade_Term_INCOTERM** - It is a international trade rule that decides who handles shipping, insurance, customs, and overall responsibility between buyer and seller.

* Container_Type
* **Carrier** - The transportation company responsible for moving the goods (like Maersk, DHL, MSC, FedEx, etc.). 

df.info()

df.describe()

df.head()

df.tail()

df.dtypes

df.describe()

df.head()

df.tail()

df.shape

### Removing unwanted columns

The column 'Shipment_ID' is not important for our analysis, It doesn't give any insights. So we are dropping that column

df=df.drop('Shipment_ID',axis=1)
df

# EDA Null Values Detection 

df.isnull().sum()

These columns contains some null values

* Quantity_Tons
* Export_Value_USD
* Shipping_Cost_USD
* Delivery_Days

<b>Checking the datatypes of the columns with null values</b>

null_cols=['Quantity_Tons','Export_Value_USD','Shipping_Cost_USD','Delivery_Days']
df[null_cols].dtypes

They are all numerical columns, So we are plotting histograms of these columns

df[null_cols].hist(color='b')

The graph is asymmetric, So we are using <mark>median</mark> to replace the null values

### Replacing the null values with median

for i in null_cols:
    df[i]=df[i].fillna(df[i].median())

df[i]

#### Checking if any null values left.

df.isnull().sum()

<b>All null values are successfully removed.</b>

# EDA Outlier Detection

Setting a variable to find out columns with numerical value only

graph=df.select_dtypes('float')
graph

Plotting a boxplot to visualise outliers

box=['Quantity_Tons','Export_Value_USD','Shipping_Cost_USD','Delivery_Days']
sns.boxplot(data=df[box])
plt.show()

for i in box:
    sns.boxplot(df[i])
    plt.show()

* We are identifying outliers in each column using for loop.

cols=['Quantity_Tons','Export_Value_USD','Shipping_Cost_USD','Delivery_Days']

for c in cols:
    Q1=np.percentile(df[c],25,method='midpoint')
    Q3=np.percentile(df[c],75,method='midpoint')
    IQR=Q3-Q1

    u=Q3+IQR*1.5
    l=Q1-IQR*1.5
    
    outliers=df[(df[c]<l) | (df[c]>u)]
    print(outliers.shape[0])

That is
* 0 outliers value for Quantity_Tons
* 20 outliers value for Export_Value_USD
* 20 outliers value for Shipping_Cost_USD
* 20 outliers value for Delivery_Days

Removing those outliers

for col in cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR
    
    df = df[(df[col] >= lower_limit) & (df[col] <= upper_limit)]

df.shape

All outliers are removed successfully 

# EDA Visualization 

We use visualizations in this project to clearly understand patterns, trends, and relationships in the export shipment data that are not obvious from raw numbers. They also help us identify anomalies, compare categories, and communicate insights quickly and effectively for decision-makin

## Line Plot – Yearly Trend of Export Value / Quantity

# Convert Date column to datetime
df["Date"] = pd.to_datetime(df["Date"])

# Extract year
df["Year"] = df["Date"].dt.year

# Group year-wise
yearly = df.groupby("Year")[["Quantity_Tons", "Export_Value_USD"]].sum().reset_index()

sns.lineplot(data=yearly, x="Year", y="Export_Value_USD", marker="o", label="Export Value (USD)")

plt.title("Yearly Trend of Quantity & Export Value")
plt.xlabel("Year")
plt.ylabel("Total Value")
plt.grid(True)
plt.show()

## Bar Plot – Commodity vs Total Export Quantity

# Group by commodity and get total quantity
commodity_qty = (
    df.groupby("Commodity")["Quantity_Tons"]
      .sum(numeric_only=True)
      .head(5)                # top 15 commodities
      .reset_index()
)

# Plot

sns.barplot(
    data=commodity_qty,
    x="Quantity_Tons",
    y="Commodity",
    palette="Greens_r",
    hue="Commodity"
)

plt.title("Top Commodities by Total Export Quantity")
plt.xlabel("Total Quantity (Tons)")
plt.ylabel("Commodity")

## Heatmap – Correlation Between Numerical Columns

num_cols = ["Quantity_Tons", "Export_Value_USD", "Shipping_Cost_USD", "Delivery_Days"]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")
corr = df[num_cols].corr()
sns.heatmap(corr,annot=True,cmap="coolwarm",fmt=".2f",linewidths=0.5, )
plt.title("Correlation Heatmap of Key Numerical Features")

## Scatter Plot – Quantity vs Export Value

df.columns

plt.figure(figsize=(8,5))
sns.regplot(data=df, x="Delivery_Days", y="Shipping_Cost_USD", line_kws={"color": "red"})
plt.title("Scatter Plot – Shipping Cost vs Delivery Days")
plt.show()

####  straightline indicates that there is no correlation between shipping cost and delivery days

plt.figure(figsize=(7,5))
sns.countplot(data=df, x="Transport_Mode",color='green',hue='Transport_Mode', palette='dark:green')   # Column containing Sea / Air / Road
plt.title("Transport Mode Distribution")
plt.xlabel("Transport Mode")
plt.ylabel("Count")
plt.show()

## Bar Plot – Top 10 Exporting Countries by Value

# Group by country and get top 10
top10 = (
    df.groupby("Country")["Export_Value_USD"]
      .sum(numeric_only=True)
      .sort_values(ascending=False)
      .head(10)
      .reset_index()
)

# Plot

sns.barplot(
    data=top10,
    x="Export_Value_USD",
    y="Country",
    hue="Country",     
    palette="Blues_r"
)

plt.title("Top 10 Exporting Countries by Export Value")
plt.xlabel("Total Export Value (USD)")
plt.ylabel("Country")
plt.show()

ab=df.groupby('Carrier')['Export_Value_USD'].mean()

ab = df.groupby('Carrier')['Export_Value_USD'].mean().reset_index()

sns.barplot(data=ab, x='Carrier', y='Export_Value_USD')
plt.title("Average Export Value by Carrier")
plt.xlabel("Carrier")
plt.ylabel("Mean Export Value (USD)")
plt.show()


ac = df.groupby('Carrier')['Shipping_Cost_USD'].mean().reset_index()

plt.figure(figsize=(8,5))
sns.barplot(data=ac, x='Carrier', y='Shipping_Cost_USD')
plt.title("Average Shipping Cost by Carrier")
plt.xlabel("Carrier")
plt.ylabel("Mean Shipping Cost (USD)")
plt.show()

df.drop(columns=['Trade_Term_INCOTERM'],inplace=True)

df.select_dtypes('object').nunique()

# Encoding

df1=pd.get_dummies(df.select_dtypes('object')).astype('int')
df1

df=pd.concat([df,df1],axis=1)

a=df.select_dtypes('object')
df.drop(columns=a,inplace=True)

df

# Scaling

for i in df:
    sns.histplot(x=df[i],kde=True)
    plt.show()

from sklearn.preprocessing import MinMaxScaler

graph=df.select_dtypes('float')

minmax= MinMaxScaler()
c=minmax.fit_transform(graph)
c

x1=pd.DataFrame(c, columns=["Quantity_Tons", "Export_Value_USD", "Shipping_Cost_USD", "Delivery_Days"])
x1

df.drop(columns=graph,inplace=True)

df=pd.concat([x1,df],axis=1)

df.describe()

# Conclusion 

## Shipment Trends
Products exported the most

The highest-exported items are the top commodities in bar chart, mainly:

* Electronics

* Automobile Parts

* Textiles

These consistently appear with the highest total Quantity_Tons.

Top export destination countries

Your Top 10 Countries by Export Value plot shows the leaders:

* United States

* United Arab Emirates

* United Kingdom

* Germany

* Singapore

These countries generate the highest export revenue.

## Delivery Performance
Average delivery time

Your dataset shows delivery times typically around:

* 12–18 days on average

* (If you want the exact mean, I can compute it.)

### What causes delays?

* Based on correlations + scatter:

* Sea transport has the highest delivery days

* Countries farther from the origin have longer delivery times

* Higher shipping cost is often linked with longer routes

* Some commodities (like machinery) tend to take longer due to inspection & clearance

## Logistics Insights
Most common transport mode

From your count plot:

* Sea is the most frequently used

* Air is second

* Road is used mainly for nearby countries

Route efficiency & cost differences

### Your data shows:

Air routes → Fastest but most expensive

Sea routes → Cheapest but slowest

Road routes → Medium cost, used regionally

Some countries show high shipping cost + long delivery days, meaning inefficient or long-distance routes

# Quick Summary

## Quantity vs Export Value (Positive Relationship)

When shipment quantity increases, export value also tends to increase — meaning larger shipments generally bring in higher revenue.

## Shipping Cost vs Delivery Days (Moderate Positive Relationship)

Shipments that take longer to deliver often have higher shipping costs, suggesting that long-distance or slower routes are also more expensive.

## Weight vs Shipping Cost (Non-linear but upward trend)

Heavier shipments usually cost more to transport, but not at a constant rate — transport mode and routes create variations, so the relationship is not perfectly linear.

